In [3]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy
import queue

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  #

W1011 08:20:52.946810 4539028928 __init__.py:690] 

  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  
W1011 08:20:54.710767 4539028928 deprecation.py:323] From <ipython-input-3-e74d6586e2ad>:6: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1011 08:20:54.711786 4539028928 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a futur

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W1011 08:20:54.930864 4539028928 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
class Config:
    def __init__(self):
        self.n_input = 784  # input layer (28x28 pixels)
        self.n_hidden1 = 512  # 1st hidden layer
        self.n_hidden2 = 256  # 2nd hidden layer
        self.n_embedding = 128  # 3rd embedding layer
        self.n_output = 10  # output layer (0-9 digits)
        self.n_model_inputs = self.n_input + self.n_embedding  # input layer (28x28 pixels)

        self.learning_rate = 1e-4
        self.learn_iterations = 1000
        self.batch_size = 128
        self.mach_batch_size = 512


class Mach:

    def __init__(self, config, mnist):
        self.c = config
        self.mnist = mnist
        self.n_train = self.mnist.train.num_examples  # 55,000
        self.n_validation = self.mnist.validation.num_examples  # 5000
        self.n_test = self.mnist.test.num_examples  # 10,000
        
        self.gradient_queue = queue.LifoQueue(maxsize=-1)

        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)
        with self.graph.as_default():
            self.input_fn()
            self.model_fn()
            self.grad_fn()
            init = tf.global_variables_initializer()
        self.session.run(init)
        
        self.child = None
        
    def input_fn(self):
        # Placeholders.
        self.X = tf.placeholder(tf.float32, [None, self.c.n_input], 'X')
        self.Y = tf.placeholder(tf.float32, [None, self.c.n_output], 'Y')
        self.C = tf.placeholder(tf.float32, [None, self.c.n_embedding], 'C')
            
    def grad_fn(self):        
        # Optimizer.
        self.optimizer = tf.train.AdamOptimizer(self.c.learning_rate)
        
        # Gradients.
        self.E_grad = tf.placeholder(tf.float32, [None, self.c.n_embedding], 'E')
        self.cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.Y, logits=self.logits))
        self.upstream_gradients = self.optimizer.compute_gradients(loss=self.cross_entropy)
        self.local_gradients = self.optimizer.compute_gradients(loss=self.cross_entropy)
        
        # Secondary metrics.
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        # Upstream Gradient placeholders.
        self.upstream_gradient_values = []
        self.upstream_placeholder_gradients = []
        for gradient_variable in self.upstream_gradients:
            grad_placeholder = tf.placeholder(tf.float32, shape=gradient_variable[1].get_shape())
            self.upstream_gradient_values.append(gradient_variable[1])
            self.upstream_placeholder_gradients.append((grad_placeholder, gradient_variable[1]))
            
        # Local Gradient placeholders.
        self.local_gradient_values = []
        self.local_placeholder_gradients = []
        for gradient_variable in self.local_gradients:
            grad_placeholder = tf.placeholder(tf.float32, shape=gradient_variable[1].get_shape())
            self.local_gradient_values.append(gradient_variable[1])
            self.local_placeholder_gradients.append((grad_placeholder, gradient_variable[1]))

        # Train steps.
        self.upstream_train_step = self.optimizer.apply_gradients(self.upstream_placeholder_gradients)
        self.local_train_step = self.optimizer.apply_gradients(self.local_placeholder_gradients)
    
    
    def model_fn(self):
        self.model_inputs = tf.concat([self.X, self.C], axis=1)
        
        weights = {
            'w1': tf.Variable(tf.truncated_normal([self.c.n_model_inputs, self.c.n_hidden1], stddev=0.1)),
            'w2': tf.Variable(tf.truncated_normal([self.c.n_hidden1, self.c.n_hidden2], stddev=0.1)),
            'w3': tf.Variable(tf.truncated_normal([self.c.n_hidden2, self.c.n_embedding], stddev=0.1)),
            'out': tf.Variable(tf.truncated_normal([self.c.n_embedding, self.c.n_output], stddev=0.1)),
        }

        biases = {
            'b1': tf.Variable(tf.constant(0.1, shape=[self.c.n_hidden1])),
            'b2': tf.Variable(tf.constant(0.1, shape=[self.c.n_hidden2])),
            'b3': tf.Variable(tf.constant(0.1, shape=[self.c.n_embedding])),
            'out': tf.Variable(tf.constant(0.1, shape=[self.c.n_output]))
        }

        # Model feature extraction.
        self.layer_1 = tf.add(tf.matmul(self.model_inputs, weights['w1']), biases['b1'])
        self.layer_2 = tf.add(tf.matmul(self.layer_1, weights['w2']), biases['b2'])
        self.E = tf.add(tf.matmul(self.layer_2, weights['w3']), biases['b3'])
        
        # Local logits.
        self.logits = tf.matmul(self.E, weights['out']) + biases['out']
        
    def Child(self, batch):
        if self.child == None:
            return numpy.zeros((numpy.shape(batch)[0], self.c.n_embedding))
        else:
            return self.child.Spike(batch)
        
    def Spike(self, spikes):
        feeds={self.X: spikes, self.C: self.Child(spikes), self.keep_prob: 1.0}
        return self.session.run([self.E], feed_dict=feeds)[0]
    
    def Grade(self, grads, spikes):
        cspikes = self.Child(spikes)
        feeds={
                self.X: batch, 
                self.C: self.Child(batch),
                self.E_grad: grads
        }
        fetches = [self.upstream_gradient_values]
        upstream_gradients = self.session.run(fetches, feeds)[0]
        self.grad_queue.put(upstream_gradients)
        
    def Perf(self):
        batch_x, batch_y = self.mnist.train.next_batch(self.c.batch_size)
        fetches = [self.cross_entropy, self.accuracy]
        feeds = {self.X: batch_x, self.Y: batch_y, self.C: self.Child(batch_x)}
        loss, accuracy = self.session.run(fetches,feeds)    
        print("Loss =", str(loss), "\t| Accuracy =", str(accuracy))

    def Train(self, n):
        for i in range(n):
            self.batch_step()

    def batch_step(self):
        batch_x, batch_y = self.mnist.train.next_batch(self.c.batch_size)
        feeds={self.X: batch_x, self.Y: batch_y, self.C: self.Child(batch_x)}
        fetches=self.local_gradients
        gradients = self.session.run(self.local_gradients, feed_dict=feeds)
        self.gradient_queue.put(gradients)

        fetches = [self.cross_entropy, self.accuracy]
        feeds = {self.X: batch_x, self.Y: batch_y, self.C: self.Child(batch_x)}
        minibatch_loss, minibatch_accuracy = self.session.run(fetches,feeds)    
        print(
            "Iteration",
            str(i),
            "\t| Loss =",
            str(minibatch_loss),
            "\t| Accuracy =",
            str(minibatch_accuracy)
            )

    def Learn(self, n):
        for i in range(n):
            self.learn_step()
        
    def learn_step(self):
        gradients = self.gradient_queue.get()
        feeds = {}
        for j, grad_var in enumerate(gradients):
            feeds[self.local_placeholder_gradients[j][0]] = gradients[j][0]
        self.session.run(self.local_train_step, feeds)
    
        
#     def Train(self):
#         # train on mini batches
#         for i in range(self.c.learn_iterations):    
            
#             # Compute gradients.
#             batch_x, batch_y = self.mnist.train.next_batch(self.c.batch_size)
#             feeds={self.X: batch_x, self.Y: batch_y, self.C: self.Child(batch_x)}
#             fetches=self.local_gradients
#             gradients = self.session.run(self.local_gradients, feed_dict=feeds)
            
#             # Now apply them.
#             # Feed batch of gradients.
#             feeds = {}
#             for j, grad_var in enumerate(gradients):
#                 feeds[self.local_placeholder_gradients[j][0]] = gradients[j][0]

#             # Apply gradients.
#             self.session.run(self.local_train_step, feeds)
                                     
#             # print loss and accuracy (per minibatch)

                
    def Test(self):
        feed_dict = {
                self.X: self.mnist.test.images, 
                self.Y: self.mnist.test.labels, 
                self.C: self.Child(self.mnist.test.images) 
        }
        test_accuracy = self.session.run(self.accuracy, feed_dict=feed_dict)
        print("\nAccuracy on test set:", test_accuracy)



In [ ]:
c = Config()
m = Mach(c, mnist)



for i in range(1000):
    m.Train(1)
    m.Learn(1)
    print (i)
    if i % 100 == 0:
        m.Perf()





Iteration 0 	| Loss = 3.203428 	| Accuracy = 0.140625
Loss = 2.9019256 	| Accuracy = 0.15625
Iteration 1 	| Loss = 2.99845 	| Accuracy = 0.1171875
Iteration 2 	| Loss = 2.843295 	| Accuracy = 0.15625
Iteration 3 	| Loss = 2.9312615 	| Accuracy = 0.140625
Iteration 4 	| Loss = 2.4517303 	| Accuracy = 0.203125
Iteration 5 	| Loss = 2.3996415 	| Accuracy = 0.234375
Iteration 6 	| Loss = 2.2242515 	| Accuracy = 0.265625
Iteration 7 	| Loss = 2.3805044 	| Accuracy = 0.2421875
Iteration 8 	| Loss = 2.383777 	| Accuracy = 0.265625
Iteration 9 	| Loss = 2.1066647 	| Accuracy = 0.2890625
Iteration 10 	| Loss = 1.8743441 	| Accuracy = 0.375
Iteration 11 	| Loss = 1.946922 	| Accuracy = 0.3125
Iteration 12 	| Loss = 1.8661624 	| Accuracy = 0.359375
Iteration 13 	| Loss = 1.5161598 	| Accuracy = 0.5078125
Iteration 14 	| Loss = 1.5732443 	| Accuracy = 0.4453125
Iteration 15 	| Loss = 1.3833759 	| Accuracy = 0.4609375
Iteration 16 	| Loss = 1.569941 	| Accuracy = 0.4453125
Iteration 17 	| Loss = 1.

Iteration 150 	| Loss = 0.39246753 	| Accuracy = 0.8828125
Iteration 151 	| Loss = 0.3590059 	| Accuracy = 0.890625
Iteration 152 	| Loss = 0.5109602 	| Accuracy = 0.8515625
Iteration 153 	| Loss = 0.38340667 	| Accuracy = 0.8671875
Iteration 154 	| Loss = 0.4706523 	| Accuracy = 0.8671875
Iteration 155 	| Loss = 0.4898553 	| Accuracy = 0.8515625
Iteration 156 	| Loss = 0.33808362 	| Accuracy = 0.8828125
Iteration 157 	| Loss = 0.5145081 	| Accuracy = 0.828125
Iteration 158 	| Loss = 0.3774789 	| Accuracy = 0.875
Iteration 159 	| Loss = 0.39137152 	| Accuracy = 0.875
Iteration 160 	| Loss = 0.36070585 	| Accuracy = 0.90625
Iteration 161 	| Loss = 0.5079918 	| Accuracy = 0.8359375
Iteration 162 	| Loss = 0.3530351 	| Accuracy = 0.8984375
Iteration 163 	| Loss = 0.50493777 	| Accuracy = 0.890625
Iteration 164 	| Loss = 0.41376936 	| Accuracy = 0.8515625
Iteration 165 	| Loss = 0.57391095 	| Accuracy = 0.8359375
Iteration 166 	| Loss = 0.4514685 	| Accuracy = 0.859375
Iteration 167 	| Los

Iteration 299 	| Loss = 0.34446466 	| Accuracy = 0.8984375
Iteration 300 	| Loss = 0.26440865 	| Accuracy = 0.9140625
Loss = 0.31822464 	| Accuracy = 0.9375
Iteration 301 	| Loss = 0.28981674 	| Accuracy = 0.9140625
Iteration 302 	| Loss = 0.28266978 	| Accuracy = 0.8984375
Iteration 303 	| Loss = 0.34305224 	| Accuracy = 0.875
Iteration 304 	| Loss = 0.3430683 	| Accuracy = 0.890625
Iteration 305 	| Loss = 0.2992872 	| Accuracy = 0.90625
Iteration 306 	| Loss = 0.58094627 	| Accuracy = 0.8359375
Iteration 307 	| Loss = 0.31132275 	| Accuracy = 0.9140625
Iteration 308 	| Loss = 0.2675435 	| Accuracy = 0.9296875
Iteration 309 	| Loss = 0.37450805 	| Accuracy = 0.8984375
Iteration 310 	| Loss = 0.4240428 	| Accuracy = 0.84375
Iteration 311 	| Loss = 0.3157211 	| Accuracy = 0.890625
Iteration 312 	| Loss = 0.4536786 	| Accuracy = 0.8828125
Iteration 313 	| Loss = 0.42648387 	| Accuracy = 0.890625
Iteration 314 	| Loss = 0.27679783 	| Accuracy = 0.921875
Iteration 315 	| Loss = 0.33828503 

Iteration 442 	| Loss = 0.3489517 	| Accuracy = 0.8984375
Iteration 443 	| Loss = 0.3342321 	| Accuracy = 0.890625
Iteration 444 	| Loss = 0.34225488 	| Accuracy = 0.8984375
Iteration 445 	| Loss = 0.3380007 	| Accuracy = 0.8984375
Iteration 446 	| Loss = 0.26514667 	| Accuracy = 0.9375
Iteration 447 	| Loss = 0.3453601 	| Accuracy = 0.8828125
Iteration 448 	| Loss = 0.30789655 	| Accuracy = 0.8984375
Iteration 449 	| Loss = 0.31518486 	| Accuracy = 0.921875
Iteration 450 	| Loss = 0.3001306 	| Accuracy = 0.921875
Iteration 451 	| Loss = 0.21328637 	| Accuracy = 0.9453125
Iteration 452 	| Loss = 0.1473242 	| Accuracy = 0.96875
Iteration 453 	| Loss = 0.36725456 	| Accuracy = 0.8984375
Iteration 454 	| Loss = 0.21360028 	| Accuracy = 0.9453125
Iteration 455 	| Loss = 0.30170822 	| Accuracy = 0.9375
Iteration 456 	| Loss = 0.46194023 	| Accuracy = 0.8359375
Iteration 457 	| Loss = 0.35217702 	| Accuracy = 0.8984375
Iteration 458 	| Loss = 0.21862645 	| Accuracy = 0.953125
Iteration 459 	

Iteration 592 	| Loss = 0.3756986 	| Accuracy = 0.8359375
Iteration 593 	| Loss = 0.389326 	| Accuracy = 0.90625
Iteration 594 	| Loss = 0.34170216 	| Accuracy = 0.8984375
Iteration 595 	| Loss = 0.3263036 	| Accuracy = 0.921875
Iteration 596 	| Loss = 0.2921024 	| Accuracy = 0.9453125
Iteration 597 	| Loss = 0.41160363 	| Accuracy = 0.8984375
Iteration 598 	| Loss = 0.38853243 	| Accuracy = 0.90625
Iteration 599 	| Loss = 0.2449511 	| Accuracy = 0.921875
Iteration 600 	| Loss = 0.29881898 	| Accuracy = 0.921875
Loss = 0.3279192 	| Accuracy = 0.890625
Iteration 601 	| Loss = 0.43256712 	| Accuracy = 0.8828125
Iteration 602 	| Loss = 0.22276533 	| Accuracy = 0.9453125
Iteration 603 	| Loss = 0.31303397 	| Accuracy = 0.890625
Iteration 604 	| Loss = 0.34874338 	| Accuracy = 0.8828125
Iteration 605 	| Loss = 0.28698885 	| Accuracy = 0.90625
Iteration 606 	| Loss = 0.3074788 	| Accuracy = 0.890625
Iteration 607 	| Loss = 0.37961742 	| Accuracy = 0.90625
Iteration 608 	| Loss = 0.27581638 	

Iteration 742 	| Loss = 0.34893578 	| Accuracy = 0.90625
Iteration 743 	| Loss = 0.21594423 	| Accuracy = 0.9296875
Iteration 744 	| Loss = 0.28216323 	| Accuracy = 0.9140625
Iteration 745 	| Loss = 0.36306486 	| Accuracy = 0.8828125
Iteration 746 	| Loss = 0.34127718 	| Accuracy = 0.8828125
Iteration 747 	| Loss = 0.25387275 	| Accuracy = 0.9609375
Iteration 748 	| Loss = 0.29192513 	| Accuracy = 0.90625
Iteration 749 	| Loss = 0.3625614 	| Accuracy = 0.8984375
Iteration 750 	| Loss = 0.30627847 	| Accuracy = 0.8828125
Iteration 751 	| Loss = 0.25094038 	| Accuracy = 0.90625
Iteration 752 	| Loss = 0.2278493 	| Accuracy = 0.9375
Iteration 753 	| Loss = 0.28174424 	| Accuracy = 0.9140625
Iteration 754 	| Loss = 0.4092275 	| Accuracy = 0.8671875
Iteration 755 	| Loss = 0.20993772 	| Accuracy = 0.953125
Iteration 756 	| Loss = 0.17751843 	| Accuracy = 0.9453125
Iteration 757 	| Loss = 0.2744519 	| Accuracy = 0.9296875
Iteration 758 	| Loss = 0.35456306 	| Accuracy = 0.8984375
Iteration 7

In [8]:
m.learn_step()

In [19]:
c = Config()
m = Mach(c, mnist)


In [21]:
for _ in range(1000):
    m2.Train()
    m.Train()
    m.Test()


Accuracy on test set: 0.9194

Accuracy on test set: 0.9234

Accuracy on test set: 0.9229

Accuracy on test set: 0.9215

Accuracy on test set: 0.9231

Accuracy on test set: 0.9204

Accuracy on test set: 0.9208

Accuracy on test set: 0.9215

Accuracy on test set: 0.9205

Accuracy on test set: 0.9217

Accuracy on test set: 0.9207

Accuracy on test set: 0.922

Accuracy on test set: 0.9194

Accuracy on test set: 0.9209

Accuracy on test set: 0.9213

Accuracy on test set: 0.9189

Accuracy on test set: 0.9192

Accuracy on test set: 0.9217

Accuracy on test set: 0.9209

Accuracy on test set: 0.9211

Accuracy on test set: 0.9211

Accuracy on test set: 0.9217

Accuracy on test set: 0.9205

Accuracy on test set: 0.9196

Accuracy on test set: 0.9214

Accuracy on test set: 0.9197

Accuracy on test set: 0.9224

Accuracy on test set: 0.9207

Accuracy on test set: 0.9202

Accuracy on test set: 0.9209

Accuracy on test set: 0.9212

Accuracy on test set: 0.9206

Accuracy on test set: 0.9217

Accuracy o

KeyboardInterrupt: 